# Download Webpages

In [30]:
import requests
import urllib.request
import bs4
from bs4 import BeautifulSoup

import uuid
import urllib.request
from tqdm import tqdm

# get user name like 'ec2-user' or 'root'
import getpass

# Generate URL of 104 Website

In [20]:
def get_url_job_list(num_pages=50):
    """
    Generate url of 104 website.
        Sample url: 
              ['https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=1&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=2&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=3&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=10&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=11&mode=s&jobsource=2018indexpoc']

    Return: return url lists
    """
    
    url_job_list = []
    
    for index in range(1, num_pages+1):
        url = 'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page={}&mode=s&jobsource=2018indexpoc'.format(index)
        url_job_list.append(url)
        
        
    return url_job_list


url_job_list = get_url_job_list()

In [53]:
url_job_list[17]

'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=18&mode=s&jobsource=2018indexpoc'

# Declare Parser Class

In [57]:
class ListParser():
    def __init__(self, url):
        
        # fetch html page and parse it.
        response = requests.get(url, timeout=10)
        self.content = BeautifulSoup(response.content, "html.parser")
        assert self.content is not None, 'Error: Empty content.'
        
        self.num_list = 0
        
        
    def get_lists(self):
        self.num_list = 0
        
        for element in self.content.findAll('article', attrs={'class': ['job-mode', 'js-job-item']}):
            if 'b-block--ad' in element.attrs['class']:
                continue
            else:
                self.num_list += 1
                yield element
        
        
class EntryParser():
    def __init__(self, element):
        """
        Args:
            element (bs4.element): element in BeautifulSoup
        """
        
        self.element = element
        
        
    def get_url(self):
        job = element.find('a', attrs={'class': 'js-job-link'})
        
        url_short = job['href']
        
        # fix invalid http link
        if 'javascript:' in url_short:
            return None
        
        
        return 'http:' + url_short
    
    
    def get_filename(self):
        
        # get a random unique id
        str_uuid = uuid.uuid4()
        
        company = element['data-cust-name'].replace('/', '-')
        title = element['data-job-name'].replace('/', '-')
        
        filename = '{}_{}_{}.html'.format(company, title, str_uuid)
        return filename
    

# Define Utility: Save Function

In [86]:
def save_page(url, filename):
    
    # workaround for long filename: OSError: [Errno 36] File name too long
    filename = filename[:150]
    
    # fix problem when file name is Chinese.
    filename = filename.encode('utf8')
    
    urllib.request.urlretrieve(url, (filename))

# Main Program

In [89]:

# if the current server is aws, the folder is not in sub-folder
folder_name = '/newvolume/topic_modeling/dataset/' if getpass.getuser() == 'ec2-user' else './dataset/'
# folder_name = './dataset/'

# for url in url_job_list[:1]:
# for url in tqdm(url_job_list[:3]):
for url in tqdm(url_job_list):
    parser = ListParser(url)
    
    for element in parser.get_lists():
        entry = EntryParser(element)
        
        print(entry.get_url())
        print(folder_name + entry.get_filename())
        
        if entry.get_url() is None:
            continue
            
        save_page(entry.get_url(), folder_name + entry.get_filename())
        
#         if parser.num_list == 2:
#             break





  0%|          | 0/50 [00:00<?, ?it/s]

http://www.104.com.tw/job/6lng1?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/現觀科技股份有限公司_Machine Learning Engineer_8f939827-6642-4d7b-95c5-549d6fe7b58d.html
http://www.104.com.tw/job/6arae?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/新馳科技有限公司_Machine Learning Technical Manager_4177128c-ac50-4f39-b18e-67300e39a763.html
http://www.104.com.tw/job/6heww?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/奔騰網路科技有限公司_Machine Learning Engineer_32bcec86-aacc-4a99-8d3e-0547fccec8a9.html
http://www.104.com.tw/job/46izo?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/沛星互動科技股份有限公司_Machine Learning Scientist_1e3d1c2e-a065-41d1-9d2d-75bab769c3fe.html
http://www.104.com.tw/job/6arac?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/新馳科技有限公司_Machine Learning Sr. Engineer_3c9d7865-1ed0-43bc-baf3-1717668cc392.html
http://www.104.com.tw/job/6mst6?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/銓鴻資訊有限公司_Deep Learning - Machine Learning Engineer (台北大安區)_





  2%|▏         | 1/50 [00:07<06:03,  7.43s/it]

http://www.104.com.tw/job/5zi1u?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/英屬開曼群島商鼎峰智能股份有限公司台灣分公司_Machine Learning Data Scientist (Taipei)_51019dd7-1d3a-4f5a-9c0d-ccce3c9a193b.html
http://www.104.com.tw/job/61rxb?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/阿福股份有限公司_電腦視覺與機器學習研究員 Computer Vision-Machine Learning Researcher_4d87be85-7008-4cf0-b36f-af5e758088f2.html
http://www.104.com.tw/job/6c3kt?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/英屬開曼群島商鼎峰智能股份有限公司台灣分公司_Machine Learning Data Scientist (Hsinchu)_0382cd3b-09ae-4057-ade3-3f7c6fc90a15.html
http://www.104.com.tw/job/6mqth?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/鴻華國際科技股份有限公司_Machine Learning - AI Application Engineer_eb002cae-361d-4218-9d64-4f4c8a62cc9e.html
http://www.104.com.tw/job/5ua2x?jobsource=jolist_a_relevance
/newvolume/topic_modeling/dataset/奇卓科技股份有限公司_Machine Learning-Deep learning 工程師_ee1c54a9-487a-493b-b76e-02f4e41b2288.html
http://www.104.com





  4%|▍         | 2/50 [00:14<05:55,  7.40s/it]

...





 96%|█████████▌| 48/50 [06:14<00:15,  7.91s/it]

http://www.104.com.tw/job/6nf9w?jobsource=jolist_b_relevance
/newvolume/topic_modeling/dataset/盛利來科技有限公司_Sr. AI Engineer - Video Application_cd9294f2-df5e-458f-bcff-2abf508479ff.html
http://www.104.com.tw/job/3xt3a?jobsource=jolist_b_relevance
/newvolume/topic_modeling/dataset/甲尚股份有限公司_商品化經理 Commercialization Manager_268f98d7-a334-409f-a75d-7f3a329243b5.html
http://www.104.com.tw/job/6mdob?jobsource=jolist_b_relevance
/newvolume/topic_modeling/dataset/光寶科技股份有限公司_演算法工程師_5bd517c3-e265-49b4-a03b-999f75ba45ad.html
http://www.104.com.tw/job/6nemf?jobsource=jolist_b_relevance
/newvolume/topic_modeling/dataset/台新金控_台新國際商業銀行股份有限公司_〔投顧〕國際產業策略研究員(台北)_e76d19f4-e30d-4145-908e-9a3e3735488b.html
http://www.104.com.tw/job/6n94v?jobsource=jolist_b_relevance
/newvolume/topic_modeling/dataset/瀚群股份有限公司_AI軟體工程師_9cf290d5-c28b-4431-b3fa-3c9772264166.html
http://www.104.com.tw/job/55bhl?jobsource=jolist_b_relevance
/newvolume/topic_modeling/dataset/Medtronic (Taiwan) Ltd._美敦力醫療產品股份有限公司_(台灣北部地區, 神經血管介入事業部) Te





 98%|█████████▊| 49/50 [06:21<00:07,  7.66s/it]

http://www.104.com.tw/job/6ft5a?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/香繼光集團繼光香香雞_香繼光股份有限公司_[板橋大遠百店]-晚班兼職計時-時薪160元起_86fc8eb3-8932-4fbf-a3e7-42788b61a2fc.html
http://www.104.com.tw/job/6eb8b?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/香繼光集團繼光香香雞_香繼光股份有限公司_[台北ATT店]-晚班兼職計時-時薪160元起_d81badd6-3d16-4782-8484-8bf578786b56.html
http://www.104.com.tw/job/6e7vb?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/香繼光集團繼光香香雞_香繼光股份有限公司_[新竹西大店]-兼職計時-時薪160元起_f5eda146-86bb-4f24-b572-e18774b75aeb.html
http://www.104.com.tw/job/666dr?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/香繼光集團繼光香香雞_香繼光股份有限公司_[台北開封店]-晚班兼職計時-時薪160元起_bfbe2b62-3be7-4bce-ad92-36b9768a61dc.html
http://www.104.com.tw/job/6b9ex?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/香繼光集團繼光香香雞_香繼光股份有限公司_新北新莊店-晚班兼職計時-時薪160元起_f113337e-1937-44b3-9012-c0c6958a5f2b.html
http://www.104.com.tw/job/6k16l?jobsource=2018indexpoc
/newvolume/topic_modeling/dataset/香繼光集團繼光香香雞_香繼光股份有限公司_[台南南紡店]-暑期兼職計時-時薪





100%|██████████| 50/50 [06:29<00:00,  7.54s/it]